# 病床姿勢監測 - LabelMe 資料準備 (多核加速版)

## 目標：
1. 載入並對齊 Thermal & RGB
2. 匯出 RGB 圖片 (JPG)
3. 準備 LabelMe 標註環境
4. 提供轉換腳本 (LabelMe JSON → 訓練格式)

In [1]:
import os
import json
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
import shutil

# 多核配置
NUM_WORKERS = max(1, mp.cpu_count() - 1)
CHUNK_SIZE = 100

# 路徑配置
BASE_DIR = Path(os.getcwd())
DATA_DIR = BASE_DIR / 'data'
THERMAL_DIR = DATA_DIR / 'thermal'
CAMERA_DIR = DATA_DIR / 'camera'
OUTPUT_TXT = CAMERA_DIR / 'output.txt'
AVI_FILE = CAMERA_DIR / 'output.avi'

# 輸出目錄
OUTPUT_DIR = BASE_DIR / 'output'
LABELME_DIR = OUTPUT_DIR / 'labelme_project'  # LabelMe 專案目錄
DATASET_DIR = OUTPUT_DIR / 'person_pose_dataset'

for d in [LABELME_DIR, DATASET_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# 參數
THERMAL_RESOLUTION = (32, 24)
TARGET_FPS = 8
TIMESTAMP_TOLERANCE = 1.0 / TARGET_FPS / 4

# 翻轉設定 (稍後調整)
FLIP_THERMAL = False
FLIP_RGB = False

print(f"🚀 多核處理: {NUM_WORKERS} workers")
print(f"📁 LabelMe 專案目錄: {LABELME_DIR}")

🚀 多核處理: 3 workers
📁 LabelMe 專案目錄: /home/gary/claude4.5/output/labelme_project


## Step 1: 載入並對齊資料 (與之前相同)

In [2]:
# (此處包含之前的所有對齊邏輯)
# - parse_output_txt
# - load_thermal_data_multicore
# - analyze_video
# - pair_thermal_rgb
# ...

# 執行對齊，得到 aligned_pairs
# aligned_pairs = [...]

print(f"\n✅ 對齊完成: {len(aligned_pairs)} 對")

NameError: name 'aligned_pairs' is not defined

## Step 2: 匯出 RGB 圖片 (LabelMe 格式)

In [ ]:
def export_for_labelme(pairs: List[Dict], avi_path: Path, output_dir: Path, flip: bool = False):
    """
    匯出 RGB 圖片供 LabelMe 標註
    """
    images_dir = output_dir / 'images'
    images_dir.mkdir(exist_ok=True)
    
    # 準備批次處理
    def export_batch(args):
        pairs_batch, avi_path, images_dir, flip = args
        cap = cv2.VideoCapture(str(avi_path))
        
        exported = []
        for pair in pairs_batch:
            cap.set(cv2.CAP_PROP_POS_FRAMES, pair['rgb_frame_idx'])
            ret, frame = cap.read()
            
            if not ret:
                continue
            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            if flip:
                frame_rgb = cv2.flip(frame_rgb, 1)
            
            # LabelMe 命名: frame_XXXXX.jpg
            filename = f"frame_{pair['pair_id'].replace('pair_', '')}.jpg"
            save_path = images_dir / filename
            
            cv2.imwrite(str(save_path), cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR),
                       [cv2.IMWRITE_JPEG_QUALITY, 95])
            
            exported.append({
                'filename': filename,
                'pair_id': pair['pair_id'],
                'timestamp': str(pair['timestamp'])
            })
        
        cap.release()
        return exported
    
    # 分批處理
    batches = [pairs[i:i+CHUNK_SIZE] for i in range(0, len(pairs), CHUNK_SIZE)]
    args_list = [(batch, avi_path, images_dir, flip) for batch in batches]
    
    print(f"\n匯出 RGB 圖片 (多核: {NUM_WORKERS} workers)...")
    all_exported = []
    with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
        results = list(tqdm(
            executor.map(export_batch, args_list),
            total=len(batches),
            desc="匯出 RGB"
        ))
    
    for result in results:
        all_exported.extend(result)
    
    # 儲存 metadata
    metadata_df = pd.DataFrame(all_exported)
    metadata_path = output_dir / 'image_metadata.csv'
    metadata_df.to_csv(metadata_path, index=False, encoding='utf-8-sig')
    
    print(f"\n✅ 匯出完成:")
    print(f"   圖片數: {len(all_exported)}")
    print(f"   位置: {images_dir}")
    print(f"   Metadata: {metadata_path}")
    
    return all_exported

# 執行匯出
exported_images = export_for_labelme(aligned_pairs, AVI_FILE, LABELME_DIR, FLIP_RGB)

## Step 3: 建立 LabelMe 類別定義

In [ ]:
# 定義標註類別
pose_classes = {
    "lying": "躺著 (在床上平躺)",
    "sitting": "坐著 (坐在床上或床邊)",
    "fallen": "跌倒 (跌落床下)",
    "empty": "空床 (無人)",
    "uncertain": "不確定 (畫面不清楚)"
}

# 儲存類別定義
classes_file = LABELME_DIR / 'classes.txt'
with open(classes_file, 'w', encoding='utf-8') as f:
    for class_name, description in pose_classes.items():
        f.write(f"{class_name}\n")

print(f"✅ 類別定義已儲存: {classes_file}")
print(f"\n標註類別:")
for name, desc in pose_classes.items():
    print(f"  - {name}: {desc}")

## Step 4: 建立 LabelMe 使用說明

In [ ]:
readme_content = f"""# LabelMe 標註指南 - 病床姿勢辨識

## 📁 專案結構
```
labelme_project/
├── images/              # RGB 圖片 ({len(exported_images)} 張)
├── annotations/         # LabelMe JSON 標註 (標註後產生)
├── classes.txt          # 類別定義
├── image_metadata.csv   # 圖片 metadata
└── README.md           # 本檔案
```

## 🚀 開始標註

### 1. 啟動 LabelMe
```bash
cd {LABELME_DIR}
labelme images --labels classes.txt --output annotations
```

### 2. 標註步驟
1. LabelMe 視窗會自動開啟
2. 點選「Create Polygons」或按 `Ctrl+N`
3. 用滑鼠框選**人體區域**
4. 從下拉選單選擇姿勢類別:
   - **lying** - 躺著
   - **sitting** - 坐著
   - **fallen** - 跌倒
   - **empty** - 空床
   - **uncertain** - 不確定
5. 按 `Ctrl+S` 儲存
6. 按 `D` 下一張圖片

### 3. 標註建議
- **框選範圍**: 盡量框到完整人體輪廓
- **模糊圖片**: 標註為 `uncertain`
- **空床**: 標註為 `empty`
- **部分身體**: 只要能判斷姿勢就標註

### 4. 快捷鍵
- `Ctrl+N`: 建立新標註
- `Ctrl+S`: 儲存
- `D`: 下一張
- `A`: 上一張
- `Ctrl+Z`: 復原

## 📊 標註進度追蹤
- 總圖片數: {len(exported_images)}
- 已標註: 執行 `ls annotations/*.json | wc -l` 查看

## ⚠️ 注意事項
1. 每張圖片只標註**一個人**
2. 如果有多人，標註**最明顯的那位**
3. 定期備份 `annotations/` 資料夾

## 🔄 標註完成後
執行轉換腳本:
```bash
python convert_labelme_to_dataset.py
```

生成時間: {datetime.now()}
"""

readme_path = LABELME_DIR / 'README.md'
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)

print(f"\n✅ 使用說明已生成: {readme_path}")

## Step 5: 建立 annotations 目錄

In [ ]:
annotations_dir = LABELME_DIR / 'annotations'
annotations_dir.mkdir(exist_ok=True)

print(f"✅ 標註目錄已建立: {annotations_dir}")
print(f"\n📝 下一步:")
print(f"\n1. 開啟終端機")
print(f"2. 執行以下指令:")
print(f"\n   cd {LABELME_DIR}")
print(f"   labelme images --labels classes.txt --output annotations")
print(f"\n3. 開始標註！")